# Bifurcation equations {#sec-20230102153124}

In this chapter, the bifurcation analysis of the perfect system is performed symbolically. The starting point is the symbolic expression of the energy $(u, \lambda) \mapsto \E(u, \lambda)$ that was derived in @sec-20230105101657.

To begin with, we perform the relevant initializations and retrieve the expression of the energy.

In [ ]:
import json
import lsk

from sympy import *

#λ, u, v, u_hat, v_hat, w_hat = sp.symbols(r"\lambda u v \hat{u} \hat{v} \hat{w}")

with open("problem_setting.out.json", "r") as f:
    data = json.load(f)
    
E = eval(data["E"])

In [ ]:
#| code-fold: true
lsk.display_latex_equation(r"\E(u, \lambda)", E)

## Outline of the strategy

In this section, we show that, besides the fundamental branch $u^\ast(\lambda)$, other (bifurcated) equilibrium branches
may pass through the critical point $(u_0, \lambda_0)$. The starting point is the characterization of an equilibrium by
the stationarity of the energy, which defines all equilibrium branches as implicit functions, which can be expanded with
respect to some perturbation parameter.

The approach adopted here relies on the Lyapunov--Schmidt decomposition of the equilibrium branch over $V$ and
$W$. Other approaches are possible, see e.g. @chak2018.

For a given value of $\lambda$ (close to the critical value $\lambda_0$), We seek an equilibrium point $u$ of the
system, such that

$$
\E_{,u}(u, \lambda; \hat{u}) = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230102025737}

In [ ]:
res = (E.diff(u) * u_hat).expand()

The above equation reads: for all $\hat{u} \in U$

In [ ]:
#| code-fold: true
display_latex_equation(res, 0)

The equilibrium state $u$ is projected onto the orthogonal subspaces $V$ and $W$.

$$
u = u^\ast(\lambda) + \xi_i \, v_i + w, \quad \text{with} \quad w \in W.
$$ {#eq-20220902174235}

It follows from the orthogonality of $V$ and $W$ that $\langle v_i, w \rangle = 0$ for all $i=1, \ldots, m$. Analysis of the bifurcated branches therefore reduces to finding $\xi_1, \ldots, \xi_m \in \reals$ and $w \in W$, such that

$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{u}] = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230107164346}

The method proceeds in three steps

1. Eq. (@eq-20230107164346) is first expressed for $\hat{u} \in W$ which leads to a definition of $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$ (see @sec-20230102030010),
2. Eq. (@eq-20230107164346) is then expressed for $\hat{u} \in V$ which leads to a definition of $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$ (see @sec-20230102030125),
3. finally, a parametrization $\eta$ of $\xi_1$, \dots $\xi_m$ is introduced and the Taylor expansion of $u$ and $\lambda$ with respect to $\eta$ is derived (see @sec-20230102030224).

## Elimination of $w$ {#sec-20230102030010}

We first test Eq. (@eq-20230102025737) with $\hat{w} \in W$

$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{w}] = 0, \quad \text{for all} \quad \hat{w} \in W.
$$ {#eq-20220825143616}

This defines $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$. The following asymptotic expansion
about $\xi_1 = \cdots = \xi_m = 0$ and $\lambda = \lambda_0$ is postulated

$$
\begin{aligned}
w(\xi_1, \ldots, \xi_m, \lambda) ={} & \xi_i \, w_i + \bigl( \lambda - \lambda_0 \bigr) \, w_\lambda\\
&+ \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij} + \tfrac{1}{2} \bigl( \lambda - \lambda_0 \bigr)^2 w_{\lambda\lambda} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
\end{aligned}
$$ {#eq-20230107172048}

where $w_i$, $w_\lambda$, $w_{ij}$, $w_{\lambda\lambda}$ and $w_{i\lambda}$ are elements of $W$ to be defined.

Indicial expressions are somewhat difficult to handle in SymPy, we therefore use a "trick"

- `ξ` stands for $\xi_i$,
- `w_ξ` stands for $w_i$,
- `w_ξξ`stands for $w_{ij}$,
- `w_ξλ` stands for $w_{i\lambda}$.

Then expressions such as `ξ * w_ξ` (resp. `ξ^2 * w_ξξ`) should be understood as $\xi_i \, w_i$ (resp. $\xi_i \, \xi_j \, w_{ij}$).

In [ ]:
ξ, w_ξ, w_λ, w_ξξ, w_ξλ, w_λλ = sp.symbols(r"\xi w_\xi w_\lambda w_{\xi\xi} w_{\xi\lambda} w_{\lambda\lambda}")
w = ξ * w_ξ + λ * w_λ + (ξ**2 * w_ξξ + 2 * ξ * λ * w_ξλ + λ**2 * w_λλ) / 2

It will be useful to define the critical point

In [ ]:
crit = {ξ: 0, λ: 0}

Eq. (@eq-20230107172048) is plugged into Eq. (@eq-20220825143616), which then delivers an asymptotic expansion of the stationarity equation. We will nullify the terms of order up to 2 in $\xi_i$ and $\lambda$.

In [ ]:
res_w = res.subs({u_hat: w_hat, u: u_star + ξ * v + w}).expand()

We can then nullify the various coefficients of this polynomial expansion, starting with the $\xi_i$ terms

In [ ]:
eq = sp.Eq(simplify_E2(res_w.coeff(ξ)), 0).subs(crit)
display(eq)

which should be understood as

$$
\E_2(w_i, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W.
$$

As argued in @sec-20230107173921, this leads to $w_i = 0$. We now consider the $\lambda$ term

In [ ]:
eq = sp.Eq(simplify_E2(res_w.coeff(λ)), 0).subs(crit)
display(eq)

Which sould be understood as

$$
\E_2(w_\lambda, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W.
$$

This again leads to $w_\lambda = 0$. The $\lambda^2$ term again delivers then vanishes automatically.

$$
\E_2(w_{\lambda\lambda}, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W,
$$

from which it results that $w_{\lambda\lambda} = 0$

In [ ]:
assert 2 * res_w.coeff(λ, 2).subs(crit).subs(w_λ, 0).expand() == E2 * w_λλ * w_hat

The $\xi_i \, \xi_j$ terms deliver the following equations

In [ ]:
(eq_w_ξξ := sp.Eq(2 * res_w.coeff(ξ, 2), 0).subs(crit).subs(w_ξ, 0).expand())

The above equation should be understood as the following variational problem: find $w_{ij} \in W$ such that

$$
\E_2(w_{ij}, \hat{w}) + \E_3(v_i, v_j, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W,
$$ {#eq-20230107180410}

which defines uniquely the $w_{ij}$. Finally, the $\xi_i \, \lambda$ terms deliver the following variational problem

In [ ]:
(eq_w_ξλ := sp.Eq(res_w.coeff(ξ).coeff(λ), 0).subs(crit).subs({w_ξ: 0, w_λ: 0}).expand())

In other words: find $w_{i\lambda} \in W$ such that

$$
\E_2(w_{i\lambda}, \hat{w}) + \dot{\E}_2(v_i, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W,
$$ {#eq-20230107180501}

which again defines $w_{i\lambda}$ uniquely.

As a conclusion, it has been shown that $w_1 = \cdots = w_m = w_\lambda = w_{\lambda\lambda} = 0$. The Taylor expansion of $w$ therefore reduces to

$$
w(\xi_1, \ldots, \xi_m, \lambda) = \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
$$

where $w_{ij}$ and $w_{i\lambda}$ are the solutions to the variational problems (@eq-20230107180410) and (@eq-20230107180501), respectively.

## Elimination of $\lambda$ {#sec-20230102030125}

We now test Eq. (@eq-20230102025737) with $\hat{v} \in V$

$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w(\xi_1, \ldots, \xi_m, \lambda), \lambda; \hat{v}] = 0, \quad \text{for all} \quad \hat{v} \in W.
$$ {#eq-20230110092313}

This now defines $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$. The following asymptotic expansion
about $\xi_1 = \cdots = \xi_m = 0$ is postulated

$$
\lambda(\xi_1, \ldots, \xi_m) = \xi_i \, \lambda_i + \tfrac{1}{2} \, \xi_i \, \xi_j \, \lambda_{ij}\\
$$ {#eq-20230110091031}

where $\lambda_i$ and $\lambda_{ij}$ are scalar coefficients to be defined.

As mentioned above, handling of indicial expressions is a bit awkward in [SymPy](https://www.sympy.org/). Therefore, we define well-formed products such as $\lambda_i \, \xi_i$ or $\lambda_{ij} \, \xi_i \, \xi_j$ as *fully parenthesized symbols*.

The auxiliary variable `t` records the order of each term with respect to the $\xi_1, \ldots, \xi_m$.

In [ ]:
t = sp.Symbol("t")

We first introduce the asymptotic expansion, of $(\xi_1, \ldots, \xi_m, \lambda) \mapsto w(\xi_1, \ldots, \xi_m, \lambda)$, $(\xi_1, \ldots, \xi_m) \mapsto \lambda(\xi_1, \ldots, \xi_m)$ and finally $(\xi_1, \ldots, \xi_m) \mapsto u(\xi_1, \ldots, \xi_m)$ see Eqs. (@eq-20230107172048), (@eq-20230110091031) and (@eq-20220902174235).

In [ ]:
w_bif = (t * λ * sp.Symbol(r"\bigl(\xi_k\,w_{k\lambda}\bigr)") 
         + t **2 / 2 * sp.Symbol(r"\bigl(\xi_i\,\xi_j\,w_{ij}\bigr)"))

λ_bif = (t * sp.Symbol(r"\bigl(\xi_{p}\,\lambda_{p}\bigr)")
         + t**2 / 2 * sp.Symbol(r"\bigl(\xi_m\,\xi_n\,\lambda_{mn}\bigr)"))

u_bif = u_star + t * sp.Symbol(r"\bigl(\xi_{l}\,v_{l}\bigr)") + w_bif

In [ ]:
#| code-fold: true
display_latex_equation(r"w(\xi_1, \ldots, \xi_m, \lambda)", w_bif)
display_latex_equation(r"\lambda(\xi_1, \ldots, \xi_m)", λ_bif)
display_latex_equation(r"u(\xi_1, \ldots, \xi_m)", u_bif)

The above expansions are plugged in the general expression of $\E_{,u}(u, \lambda; \bullet)$ and tested with $\hat{v} \in V$, see Eq. (@eq-20230110092313).

In [ ]:
res_v = res.subs(u_hat, v_hat).subs(u, u_bif).subs(λ, λ_bif).expand()

We used three consecutive `subs()` rather than one big dictionary substitution for better control on the *order* of the substitutions. Indeed, we must make sure `λ_bif` is also plugged into `u_bif`. 

We get an asymptotic expansion of the residual with respect to $\xi_1, \ldots, \xi_m$. This residual must vanish for all $\xi_i$ sufficiently small. Therefore, all coefficients of the expansion `res_v` must be nullified at all orders and for all $\xi_i$ sufficiently small. Consiering the expansion up to third order, we get after simplification only *two* bifurcation equations, since the terms of order 0 and 1 are identically null.

In [ ]:
coeffs = [simplify_E2(res_v.coeff(t, k)) for k in range(4)]
    
assert coeffs[0] == 0
assert coeffs[1] == 0

In [ ]:
#| code-fold: true
for coeff in coeffs[2:]:
    display_latex_equation(coeff, 0)

The above bifurcation equations should be understood as follows
$$
\xi_j \, \xi_k \, \bigl[
\tfrac{1}{2} \E_3(v_j, v_k, \hat{v})
+ \lambda_k \, \dot{\E}_2(v_j, \hat{v})
\bigr] = 0
$$ {#eq-20230115101130}
and
$$
\begin{aligned}[b]
\xi_j \, \xi_k \, \xi_l \, \bigl[
\tfrac{1}{2} \E_3(v_j, w_{kl}, \hat{v})
+ \lambda_l \, \E_3(v_j, w_{k\lambda}, \hat{v})
+ \tfrac{1}{6} \E_4(v_j, v_k, v_l, \hat{v}) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_2(w_{jk}, \hat{v})
+ \lambda_k \, \lambda_l \, \dot{\E}_2(w_{j\lambda}, \hat{v})
+ \tfrac{1}{2} \lambda_{kl} \, \dot{\E}_2(v_j, \hat{v}) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_3(v_j, v_k, \hat{v})
+ \tfrac{1}{2} \lambda_k \, \lambda_l \, \ddot{\E}_2(v_j, \hat{v})
\bigr] &= 0.
\end{aligned}
$$ {#eq-20230115101147}

Note that these equations are in fact *asymptotic expansions*. As such, they should hold for all $\xi_1, \ldots, \xi_m$ sufficiently small. Since both equations are homogeneous in $\xi_1, \ldots, \xi_m$, it is equivalent to require these to hold for all $\xi_1, \ldots, \xi_m \in \reals$. 